In [1]:
import pandas as pd

In [2]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [3]:
path = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S2to5combined_20ul_40min.xlsx'
df_mock = load_data(path)
path = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S6to10combined_20ul_40min.xlsx'
df_rsv = load_data(path)
df_mock.shape, df_rsv.shape

((4593, 5), (2727, 5))

In [4]:
path = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S2to5combined_20ul_9min.xlsx'
df_mock_ = load_data(path)
path = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S6to10combined_20ul_9min.xlsx'
df_rsv_ = load_data(path)
df_mock_.shape, df_rsv_.shape

((1658, 5), (1672, 5))

In [16]:
df_glu_5p = pd.read_excel('~/Studio/tools/glu_24203_5p_6mG.xlsx')
df_glu_3p = pd.read_excel('~/Studio/tools/glu_24203_3p_6mG.xlsx')

In [12]:
df_ngs_5p = pd.read_excel('~/Studio/tools/glu_bao_5p.xlsx')
df_ngs_3p = pd.read_excel('~/Studio/tools/glu_bao_3p.xlsx')

In [7]:
df_ngs_src = pd.read_excel('~/Downloads/EJ_summary.xlsx')

df_ngs_src = df_ngs_src.drop_duplicates(subset=['seqs'])
df_ngs_src.shape

(12, 14)

In [11]:
ngs = df_ngs_src.seqs[0]
ngs += 'CCA'
ngs = ngs.replace('T', 'U')
ngs

'UCCCUGGUGGUCUAGUGGUUAGGAUUCGGCGCUCUCACCGCCGCGGCCCGGGUUCGAUUCCCGGUCAGGGAACCA'

In [ ]:
for idx, row in df_ngs_src.iterrows():
    if idx != 5:
        continue
    seq = row['seq']
    df_5p = pd.read_json(row['MassLadder5p'])
    df_3p = pd.read_json(row['MassLadder3p'])
    
    df_5p['Base'] = list(seq)
    df_3p['Base'] = list(seq[::-1])
    
    df_5p['Idx'] = np.array(df_5p.index)+1
    df_3p['Idx'] = 75-np.array(df_3p.index)
    print(row)
#     break


In [47]:
# df_sample = df_mock.copy()
dft = df_glu_3p.copy()
dft.Mass += 14
dfm_3p = match_dfs(dft, df_rsv)
plotly_zone(dfm_3p, y='Vol')
# dft.Mass += M
dft.Mass -= 1
dfm_3p_me = match_dfs(dft, df_rsv)
plotly_zone(dfm_3p_me, y='Vol')

In [ ]:
df_ngs = df_3p.copy()

df_ngs = pd.read_excel('~/Studio/seq_bank/data/glu/Glu-CCC_5p.xlsx')
seq = 'UCCCUGGUGGUCUAGUGGUUAGGAUUCGGCGCUCCCACCGCCGCGGCCCGGGUUCGAUUCCCGGUCAGGGAACCA'
ori, ngs_seq = 5, seq
df_ngs['Base'] = list(ngs_seq)
if ori == 5:
    df_ngs['Idx'] = np.array(df_ngs.index)+1
else:
    df_ngs['Idx'] = 75-np.array(df_ngs.index)
df_ngs.head()

In [ ]:
# remove adducts from df_glu
adducts = [K, Na]
dfm_adducts_list = list()
for adduct in adducts:
    dfm_r = match_dfs(df_mock, df_mock, shift=adduct)
    dfm_adducts_list.append(dfm_r)
dfm_adducts = pd.concat(dfm_adducts_list).drop_duplicates()
df_mock_without_adducts = df_mock.drop(dfm_adducts.index)
df_mock_adducts_only = dfm_adducts.copy()
print(df_mock.shape[0], df_mock_without_adducts.shape[0])

dfm_adducts_list = list()
for adduct in adducts:
    dfm_r = match_dfs(df_rsv, df_rsv, shift=adduct)
    dfm_adducts_list.append(dfm_r)
dfm_adducts = pd.concat(dfm_adducts_list).drop_duplicates()
df_rsv_without_adducts = df_rsv.drop(dfm_adducts.index)
df_rsv_adducts_only = dfm_adducts.copy()
print(df_rsv.shape[0], df_rsv_without_adducts.shape[0])

In [ ]:
plotly_zone(df_rsv_adducts_only)

In [ ]:
# df_wild, df_alkb = df_, df_gln1_alkb_without_adducts
df_mock_sample = df_mock.copy()
df_rsv_sample = df_rsv.copy()
df_mock_sample.shape, df_rsv_sample.shape

In [ ]:
df_mock_adducts, df_rsv_adducts = df_mock_adducts_only, df_rsv_adducts_only
df_mock_adducts.shape, df_rsv_adducts.shape

In [ ]:
import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

def func(df_s1, df_s2, shift):
    dfm = match_dfs(df_s1, df_s2, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

PROCESSES = 8
params = [(df_ngs, df_mock_sample, shift) for shift in np.arange(0, 160, 1)]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits = pool.starmap(func, params)

# df_shift_hits = pd.DataFrame(shift_hits)

print('Done for regular sample.')
params = [(df_ngs, df_mock_adducts, shift) for shift in np.arange(0, 160, 1)]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits_adducts = pool.starmap(func, params)
print('Done for adducts sample.')

df_shift_hits = pd.DataFrame(shift_hits)
df_shift_hits_adducts = pd.DataFrame(shift_hits_adducts)

In [ ]:
fig = px.scatter(df_shift_hits[df_shift_hits.hit<df_shift_hits.shape[0]], x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'})
fig.show()
fig = px.scatter(df_shift_hits_adducts[df_shift_hits_adducts.hit<df_shift_hits_adducts.shape[0]], x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'})
fig.show()

In [ ]:
df_sh_top = df_shift_hits[df_shift_hits.hit>=9]
shifts = df_sh_top['shift']
df_sh_top_adducts = df_shift_hits_adducts[df_shift_hits_adducts.hit>=5]
shifts_adducts = df_sh_top_adducts['shift']
shifts.shape, shifts_adducts.shape

In [ ]:
shifts = [0, 14, 28]
shifts_adducts = [0, 14]

In [ ]:
# find out data points which can generate base calling
def func(df_s1, df_s2, shift):
    _, df = peer_dfs(df_s1, df_s2, shift=shift)
    
    bcr = base_calling_random(df, base_only=True)
    df = bcr[0].copy()
    df['Shift'] = shift
    return df

PROCESSES = 8
params = [(df_ngs, df_mock_sample, shift) for shift in shifts]
with multiprocessing.Pool(PROCESSES) as pool:
    df_basecallings = pool.starmap(func, params)
print('Done for regular sample.')
params = [(df_ngs, df_mock_adducts, shift) for shift in shifts_adducts]
with multiprocessing.Pool(PROCESSES) as pool:
    df_basecallings_adducts = pool.starmap(func, params)
print('Done for adducts sample.')

df_hits = pd.concat(df_basecallings).drop_duplicates()
df_hits_adducts = pd.concat(df_basecallings_adducts).drop_duplicates()
df_hits.shape, df_hits_adducts.shape

In [ ]:
plotly_zone(df_hits, y='Shift')
plotly_zone(df_hits_adducts, y='Shift')
df_hits.shape, df_hits_adducts.shape

In [ ]:
dft = df_hits[df_hits.Shift.isin([
                                0.0, 14.0, 28.0, 28.1, 
                                    42.0, 42.1, 
                                    #56 # 70.0, 70.1, 70.2,
                                  #37.9, 38, 52, 80
                                 ])]
dft_adducts = df_hits_adducts[df_hits_adducts.Shift.isin([
                                  22, 36, 50, 
                                    38, 52, 66, 80
                                 ])]
# print(dft.shape)
# plotly_zone(dft)
bcr = base_calling_random(dft)
plotly_basecalling(*bcr)

plotly_zone(dft, y='Shift')
plotly_zone(dft, y='Vol')
# bcr = base_calling_random(dft_adducts)
# plotly_basecalling(*bcr)
# dft_joint = pd.concat([dft, dft_adducts]).drop_duplicates()
# bcr = base_calling_random(dft_joint)
# plotly_basecalling(*bcr)